In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

from oneDtoyTask.oneDtoyTask import ToyTask, OneDfield, TwoWayComChannel
from agents.agent_Aria import Aria

from utils import getReturns, Controller

In [ ]:
epochs = 100
opt_params = {"lr":0.0005, "batch_size":8}
agent0 = Aria(opt_params=opt_params)
agent1 = Aria(opt_params=opt_params)
np.random.seed(1)
field = OneDfield(speed=10)
Task = ToyTask(field=field, observationMappingFct=lambda x: (x>0.5).astype(int), comChannel=TwoWayComChannel())



In [ ]:
obs, downlink_msgs = Task.reset()
losses = []
for epoch in range(epochs):
    a0, m0u =  agent0.select_action(obs[0], downlink_msgs[0])
    a1, m1u = agent1.select_action(obs[1], downlink_msgs[1])
    
    (obs_, downlink_msgs), r, done = Task.step([a0, a1], [m0u, m1u])
    loss0 = agent0.train_on_batch(obs[0], r[0])
    loss1 = agent1.train_on_batch(obs[1], r[1])
    obs = obs_
    losses.append([loss0, loss1])